<a href="https://colab.research.google.com/github/dylandeppiesse/realty-project/blob/main/RealtyDataNewWest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
from requests import get
import time as t
from random import randint
import pandas as pd
import itertools
import re

In [2]:
# setting headers which will make it look like this is coming
# from an actual browser instead of a scraper
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML,\
            like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [6]:
# I don't think we need this?
realtor = 'https://www.rew.ca/properties/areas/new-westminster-bc'
response = get(realtor, headers=headers)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [48]:
# setting up lists to form our dataframe later on
address = []
city = []
price = []
bed_bath_sqft = []
property_type = []

In [9]:
# time to scrape!
%%time 

n_pages = 0
for page in range(1,30):
    listing_url = 'https://www.rew.ca/properties/areas/new-westminster-bc/page/' + str(page)
    r = get(listing_url, headers=headers)
    listing_html = BeautifulSoup(r.text, 'html.parser')
    listing_container = listing_html.find_all('div', class_ = 'displaypanel-content')

    if listing_container != []:
        for container in listing_container:

            #address
            location = container.find_all('div', class_ = 'displaypanel-section')[0].text
            address.append(location)

            #city
            zone = container.find_all('ul', class_ = 'l-pipedlist displaypanel-info')
            city.append(zone)

            #price
            amt = int(container.find('div', class_ = 'displaypanel-title').text[2:].replace(',',''))
            price.append(amt)

            #Beds, Baths, and Square Footage
            living_space = container.find_all('ul', class_ = 'l-pipedlist')[1].text
            bed_bath_sqft.append(living_space)

            #property type
            p_type = container.find_all('div', class_ = 'displaypanel-info')[0].text
            property_type.append(p_type)

    else:
        break

    t.sleep(randint(1,3))

print('Scraped {} listings'.format(len(address)))

Scraped 0 listings
CPU times: user 216 ms, sys: 5.49 ms, total: 222 ms
Wall time: 899 ms


In [55]:
# this is our test area, we need to add all listings to the listing_container, currently it just adds the last page in the range
# I think there should only be the same number of addresses as listing containers, not close to 5,000 as we are getting

%%time 
listing_container = []
for page in range(1,30):
    listing_url = 'https://www.rew.ca/properties/areas/new-westminster-bc/page/' + str(page)
    r = get(listing_url, headers=headers)
    listing_html = BeautifulSoup(r.text, 'html.parser')
    listing_container += listing_html.find_all('div', class_ = 'displaypanel-content')

    if listing_container != []:
        for container in listing_container:

            #address
            location = container.find_all('div', class_ = 'displaypanel-section')[0].text
            address.append(location)

    else:
        break

        t.sleep(randint(1,3))

print('Scraped {} listings'.format(len(address)))

Scraped 9628 listings
CPU times: user 3.11 s, sys: 42.9 ms, total: 3.15 s
Wall time: 10.4 s


In [56]:
print(len(listing_container))
print(len(address))

187
9628


In [66]:
print(listing_container[:1][0])

<div class="displaypanel-content">
<div class="displaypanel-body">
<a href="/properties/3767959/1903-1235-quayside-drive-new-westminster-bc?page=1&amp;search_params%5Bquery%5D=New+Westminster%2C+BC&amp;searchable_id=189&amp;searchable_type=Geography?page=1&amp;search_params%5Bquery%5D=New+Westminster%2C+BC&amp;searchable_id=189&amp;searchable_type=Geography" title="1903-1235 Quayside Drive, New Westminster, BC, V3M 6T8"><div class="pull-right hidden-xs">
<div class="statustag statustag--gray">Featured</div>
</div>
<div class="displaypanel-title hidden-xs">
$819,000
</div>
<div class="displaypanel-section" data-tracking-url="https://www.rew.ca/track_click?business_entity_id=3767959&amp;event_name=log_listing_click&amp;page_code=lsrp&amp;widget_code=lsrc">
1903 1235 Quayside Drive
<ul class="l-pipedlist displaypanel-info">
<li>New Westminster Quay</li>
<li>New Westminster</li>
</ul>
</div>
<div class="displaypanel-section clearfix">
<ul class="l-pipedlist"><li>2 bd</li>
<li>3 ba</li>
<li